<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

### **Laboratorio de extracción de datos**

En este laboratorio, extraerá datos sin procesar de archivos JSON.

#### **Objetivos de aprendizaje**
Al finalizar este laboratorio, deberá ser capaz de
- Registrar una tabla externa para extraer datos de archivos JSON

#### **Run Setup**

Ejecute la siguiente celda para configurar variables y conjuntos de datos para esta lección.

In [ ]:
%run ../Includes/Classroom-Setup-02.3L

#### **Visión general de los datos**

Trabajaremos con una muestra de datos Kafka en bruto escritos como archivos JSON. 

Cada archivo contiene todos los registros consumidos durante un intervalo de 5 segundos, almacenados con el schema completo de Kafka como un archivo JSON de múltiples registros. 

El schema para la tabla:

| field  | type | description |
| ------ | ---- | ----------- |
| key    | BINARY | The **`user_id`** field is used as the key; this is a unique alphanumeric field that corresponds to session/cookie information |
| offset | LONG | This is a unique value, monotonically increasing for each partition |
| partition | INTEGER | Our current Kafka implementation uses only 2 partitions (0 and 1) |
| timestamp | LONG    | This timestamp is recorded as milliseconds since epoch, and represents the time at which the producer appends a record to a partition |
| topic | STRING | While the Kafka service hosts multiple topics, only those records from the **`clickstream`** topic are included here |
| value | BINARY | This is the full data payload (to be discussed later), sent as JSON |

#### **Extraer eventos sin procesar (raw events) de archivos JSON**
Para cargar estos datos en Delta correctamente, primero tenemos que extraer los datos JSON utilizando el schema correcto.

Cree una tabla externa con los archivos JSON ubicados en la ruta de archivo que se indica a continuación. Nombre esta tabla **`events_json`** y declare el schema anterior.

In [ ]:
%sql
CREATE TABLA IF NOT EXISTS events_json (
  key BINARY,
  offset BIGINT,
  partition INT,
  timestamp BIGINT,
  topic STRING,
  value BINARY)
USING JSON
LOCATION "${DA.paths.kafka_events}" 

**NOTA**: Usaremos Python para realizar comprobaciones ocasionalmente a lo largo del laboratorio. La siguiente celda devolverá un error con un mensaje sobre lo que hay que cambiar si no has seguido las instrucciones. Si no hay salida de la ejecución de la celda significa que has completado este paso.

In [ ]:
%python
assert spark.table("events_json"), "Table named `events_json` does not exist"
assert spark.table("events_json").columns == ['key', 'offset', 'partition', 'timestamp', 'topic', 'value'], "Please name the columns in the order provided above"
assert spark.table("events_json").dtypes == [('key', 'binary'), ('offset', 'bigint'), ('partition', 'int'), ('timestamp', 'bigint'), ('topic', 'string'), ('value', 'binary')], "Please make sure the column types are identical to those provided above"

total = spark.table("events_json").count()
assert total == 2252, f"Expected 2252 records, found {total}"

In [ ]:
%sql
SELECT * FROM events_json

<center><img src="https://i.postimg.cc/0jfNx1py/db356.png"></center>

Ejecute la siguiente celda para eliminar las tablas y archivos asociados a esta lección.

In [ ]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>